In [2]:
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
from sklearn.decomposition import PCA


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/laurenshores/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/laurenshores/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/laurenshores/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
dff = pd.read_csv("AmzingRceTeams_Data.csv")

In [4]:
dff.shape

(313, 34)

In [5]:
def preprocess(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)

dff['cleaned_occupation_x'] = dff['Occupation_x'].apply(preprocess)
dff['cleaned_occupation_y'] = dff['Occupation_y'].apply(preprocess)

In [6]:
dff

,Age_x,Age_y,LGBT_team,AficanAm_team,Friend_team,Family_team,Stranger_team,Female_team,Male_team,Coed_team,...,State_y,Same_State,Occupation_x,Occupation_y,profile_txt,profile_txt2,Top_3,Y,cleaned_occupation_x,cleaned_occupation_y
0,28,28,0,0,0,0,0,0,0,0,...,Connecticut,True,Military folk,Military folk,"[""I'd most like to receive a letter from: Nobo...",['no profile info'],0,0,military folk,military folk
1,28,27,0,0,1,0,0,1,0,0,...,Texas,True,Teachers,Teachers,"[""I'd most like to receive a letter from: Caro...",['no profile info'],0,0,teacher,teacher
2,43,42,0,0,1,0,0,1,0,0,...,Maryland,False,Missing,Missing,"[""I'd most like to receive a letter from: My k...",['no profile info'],0,0,missing,missing
3,65,60,0,0,0,0,0,0,0,1,...,Texas,True,Air Force Pilot,Retired,"[""I'd most like to receive a letter from: My k...",['no profile info'],0,0,air force pilot,retired
4,32,27,0,0,0,0,0,0,0,1,...,California,True,Actor,Bartender,"[""I'd most like to receive a letter from: Brot...",['no profile info'],0,0,actor,bartender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,38,38,0,0,0,0,0,0,0,1,...,California,True,Special Education Teacher,Hospitality Account Manager,[],['no profile info'],0,0,special education teacher,hospitality account manager
309,54,28,0,0,0,1,0,0,0,1,...,Texas,True,Paint Contractor/ Speech Pathologist,Private Flight Attendant/Influencer,['no profile info'],['no profile info'],0,0,paint speech pathologist,private flight
310,48,25,0,0,0,1,0,0,1,0,...,New York,False,Teaching Assistant/Assistant Football Coach,Senior Manager of client strategy analytics,['Notes:\n\n1: Leg 7 featured two Roadblocks. ...,['no profile info'],1,0,teaching football coach,senior manager client strategy analytics
311,42,43,0,0,1,0,0,0,1,0,...,Idaho,True,Auto Claims Training Specialist,Delivery Driver,['Notes:\n\n1: Leg 7 featured two Roadblocks. ...,['no profile info'],1,0,auto claim training specialist,delivery driver


In this notebook, I want to focus on harnessing text from surveys and Q/A's completed by the contestants. My hope is that this text will reveal certain traits or characteristics about the teams/ their skills and personalities, that could be used to enhance the model to predict winners.

First, I had to acknowledge that I did not have profile data for every team. And then to make a decision if what I had was enough to even move forward with this effort.

The other thing I had to acknowledge is that there were several different formats for this profile text, and also some noisy text that was more of a summary of the race than contestant responses. With this in mind (how noisy and inconsistent this field was), I had to think about if moving forward we be worth the effort.

Lastly, if I did decide to pursue this, what would be the way to go about wringing information from this data?

After thinking about different possible approaches to getting some meaning out of the data including: sentiment analysis, topic modeling, and text summarization, I realized that none of these would readily put me on the path to creating features that to go into my prediction model.

I decided that the best thing to do is think about some information I'd like to know, extract that information where available, and then create a feature around it. For example: 
* What are the contestants strengths?
* What are their weaknesses?
* How is their relationship with their partner? (how long, do they butt heads, is it complimentary)
* What is their personality/disposition? (optimisitic, hard working, relentless, used to getting their own way, competitive, etc)?

Of course these need to be fined tuned. But I think coming up with a few questions that center around a desired feature, and then trying to glean this info from the text may be the best approach. So maybe building a Q/A model is the right way to go about this. HOW HARD WOULD IT BE FOR ME TO BUILD A T5 MODEL? WOULD BERT SUFFICE HERE? ARE THERE PRE-TRAINED MODELS I CAN JUST USE?

types of response formatting
 * profile1 is [] or '['no profile info']
  * starts with "I'd most like to receive a letter from:"
  * starts with "Notes:
   * if Notes, then actual answers are in profile2 OR there may not be a response
  * Starts with 'Name:, then it's a short description
 * profiles with just newline characters and list brackets. Clean these out first. ['\n,'\n']
  * nothing in profile2 either. just missing data

In [ ]:
#dff[ ( dff['profile_txt'] != "[]") & (dff['profile_txt'] != "['no profile info']")]['profile_txt'].tail(20) #"['no profile info']"
#dff[  "Notes" in dff['profile_txt'] ]['profile_txt']

In [149]:
dff.iloc[3,-6] 

'["I\'d most like to receive a letter from: My kids\\nComfort item from home I\'ll miss the most: Nothing\\nHobby or activity I\'ll miss the most: Flying\\nI\'d most like to receive a phone call from: NASA, with a space slot\\nSunday paper I read most often: Dallas Morning News\\nFavorite TV show: History Channel\\nFavorite Movie: Flying Tigers\\nFavorite Actor: Alan Ladd\\nFavorite Actress: Meg Ryan\\nFavorite Music: Country & Western\\n", "I\'d most like to receive a letter from: President Bush\\nComfort item from home I\'ll miss the most: Total bathroom facility\\nHobby or activity I\'ll miss the most: Flying (as a pilot)\\nI\'d most like to receive a phone call from: Our grandbabies\\nSunday paper I read most often: Dallas Morning News\\nFavorite TV show: Ally McBeal\\nFavorite Movie: Auntie Mame\\nFavorite Actor: Mel Gibson\\nFavorite Actress: Michelle Pfeiffer\\nFavorite Music: 60s and 70s sounds\\n", \'Dave & Margaretta are the fifth team introduced at the Starting Line at the B

In [150]:
# Clean up the text to see how many samples we actually have
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Initialize lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

profile_df = dff[['Top_3', 'Y', 'profile_txt', 'profile_txt2']]
def replace_if_starts_with(original_string, check_string, replacement_string):
    if original_string.startswith(check_string):
        return replacement_string
    return original_string


def preprocess_text(text):
    
    #use regex to get rid of any text after ""After Phil Keoghan officially starts..."
    text = re.sub(r"After Phil Keoghan officially starts.*", '', text)
    
    text = text.replace("\\n", " ")
    # Remove non-alphanumeric characters
    #text = re.sub(r'\W', ' ', text)
    
    # alpha chars
    text = re.sub(r'&amp', ' ', text)
    text = re.sub('[^a-zA-Z]+', ' ', text)
    
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stop words and lemmatize
    tokens = [word.lower() for word in tokens] #if word.lower() not in stop_words]
    mystr = ' '.join(tokens)
    
    mystr = replace_if_starts_with(mystr.strip(), "no profile info", "")
    mystr = replace_if_starts_with(mystr, "note", "")
    mystr = mystr.strip()
    
    return mystr
profile_df['p1'] = profile_df['profile_txt'].apply(lambda x: preprocess_text(x))
profile_df['p2'] = profile_df['profile_txt2'].apply(lambda x: preprocess_text(x))
profile_df

/var/folders/hd/cqjrh15x0097m3t8zb9vmvnr0000gn/T/ipykernel_4756/3458273952.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profile_df['p1'] = profile_df['profile_txt'].apply(lambda x: preprocess_text(x))
/var/folders/hd/cqjrh15x0097m3t8zb9vmvnr0000gn/T/ipykernel_4756/3458273952.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profile_df['p2'] = profile_df['profile_txt2'].apply(lambda x: preprocess_text(x))


Top_3  Y  \
0        0  0   
1        0  0   
2        0  0   
3        0  0   
4        0  0   
..     ... ..   
308      0  0   
309      0  0   
310      1  0   
311      1  0   
312      1  1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [151]:
profile_df['profile'] = np.where(profile_df['p1'] == '', profile_df['p2'], profile_df['p1'] )
profile_df

/var/folders/hd/cqjrh15x0097m3t8zb9vmvnr0000gn/T/ipykernel_4756/2391527092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profile_df['profile'] = np.where(profile_df['p1'] == '', profile_df['p2'], profile_df['p1'] )


Top_3  Y  \
0        0  0   
1        0  0   
2        0  0   
3        0  0   
4        0  0   
..     ... ..   
308      0  0   
309      0  0   
310      1  0   
311      1  0   
312      1  1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [152]:
z = profile_df[profile_df['profile'] != '']
print("Number of profiles with text: " + str(len(z)))
print("-------------- Winner breakout for teams with profile txt----------------")
print(z['Y'].value_counts())

print("-------------- Top 3 breakout for teams with profile txt----------------")
print(z['Top_3'].value_counts())

Number of profiles with text: 200
-------------- Winner breakout for teams with profile txt----------------
0    180
1     20
Name: Y, dtype: int64
-------------- Top 3 breakout for teams with profile txt----------------
0    145
1     55
Name: Top_3, dtype: int64


In [153]:
pd.set_option('display.max_colwidth', None)
z.iloc[0:50,-1]              

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

# BERT

In [155]:
from transformers import pipeline

# Load the question-answering pipeline
qa_pipeline = pipeline("question-answering")


ModuleNotFoundError: No module named 'transformers'